# Programming exercises in Simulations, Machine Learning and Statistics.

One way to learn a new programming languages is to replicate classical problems from statistics or mathematics. In this notebook you'll be asked to perform 3 such exercises:

**A. Approximating $\pi$ using Monte Carlo simulations in Julia,**

**B. Demonstrating the curse of dimentionality in Machine Learning,**

**C. Constructing a statistical test indicating a trully superior player of table football match.**

## A. Approximating $\pi$ using Monte Carlo simulations in Julia 

The very first such exercise is to calculate numerically the approximate value of the number $\pi$ according to the algorithm given on [Wikipedia](https://en.wikipedia.org/wiki/Monte_Carlo_method#Overview). Following code will demontrate how to:
1. Define a function $\pi(n)$ that returns an approximation of $\pi$ for a number of simulations equal to $n$,
2. Run $\pi(n)$ function for: $10 ^ k$ number of simulations where $k\in [1, 2, 3, 4, 5, 6]$,
3. Plot the estimate of $\pi$ against the number of simulations,
4. For $k\in [1, 2, 3, 4, 5, 6]$ run $10000$ simulations each and calculate the $5\%$ and $95\%$ percentiles and averages. Draw a graph showing how these percentiles and the average change with $k$.

Following solutions were developed to use as few letters of code as possible. Thefore, the code readabilty might be partially lost. Check, if you could come up with maybe even shorter code.

We start with loading necessary packages:


In [ ]:
using StatsBase
using Plots

### A.1. Defining a function $\pi(n)$ that returns an approximation of $\pi$ for a number of simulations equal to $n$

Function $\pi(n)$ can be easily defined by a following 1-liner as below:

In [ ]:
π(n)=4*mean(sqrt.(sum(rand(n,2).^2,dims=2)).<1)

Let's deconstruct what's happending inside the body of function $\pi(n)$:
- `rand(n,2)` simulates a $n \times 2$ matrix populated with random numbers generated from uniform distribution from $0$ to $1$, i.e. $\sim U(0,1)$, Those two columns represent $(x,y)$ points disbriuted equally on the square $1 \times 1$,
- `.^2` is broadcasting (element-wise) operator for an exponent to the power of $2$ resulting in the pair $(x^2, y^2)$,
- `sum(A::AbstractArray; dims)' - sums elements of an array over the second dimension, resulting in $x^2 + y^2$,
- 'sqrt()` function taking the square of $x^2 + y^2$ and resulting in $\sqrt{x^2 + y^2}$ interpreted as a distance from $(0, 0)$,
- `.<` is a broadcasting operator of $<$ relationship returning logical values indicating whether randomly drawn $(x, y)$ belongs to the within of a circle,
- `mean()` function calculate an average over logical values by assuming `false` to be $0$ and `true` to be $1$, interpreted as $\frac{1}{4}$ of circle area,
- $4\times$ multiplication of the `mean` results in approximated whole cirle area (`mean` provided with just a $\frac{1}{4}$ of circle area).

### A.2. Running $\pi(n)$ function for: $10 ^ k$ number of simulations where $k\in [1, 2, 3, 4, 5, 6]$

In [ ]:
k=6
approx_πs = π.(10 .^(1:k))

- we start within initializing `k` to `6` by `k=6`, 
- then we create a `k`-element vector consisting of our function results. Note that our $\pi$ is followed by dot symbol `.` to denote the fact that this function is run sequentially for each element of input vector spererately. By analogy, exponent symbol `^` is proceeded by a dot `.`. Arithmetic counter from $1$ to `k` is generated by `1:k`.

### A.3. Ploting the single estimates of $\pi$ against the number of simulations

In [ ]:
plot(1:k, approx_πs,
    label = "Approx. π using 10ᵏ sim.",
    xlabel = "k (10ᵏ number of simulations)",
    ylabel = "True / Approximated π")
hline!([1, k],[pi, pi], label = "π")

Try to run this and preceeding cell a few time. You should notice that the right-hand (for $k \geq 4$) part of blue lines is relativaly stable between runs, while left-hand ($k\leq3$) side more random. It's due to higher variance of approximation for smaller sample sizes, as we'll demonstrate in a next plot.

### A.4. Drawing a graph showing how percentiles and  average change with $k$

In [ ]:
sim_n = 10 ^ 3

In [ ]:
sims = [π(10^j) for i in 1:sim_n,  j in 1:k]

In [ ]:
statistics=[[mean(sims[:,j]),
            percentile(sims[:,j], 5),
            percentile(sims[:,j], 95)] for j in 1:k]

Based on above results, we can interfere that for instance:
- there's a high risk of $10\%$ that $\pi$ approximated on 10 drawings will be higher than $4$ or lower than $2.4$,
- $100000$ simulations is enough to achieve $+/-0.01$ accuracy of $\pi$ approximation $90\%$ times
- $1000$ simulations is enough to achieve $+/-0.1$ accuracy of $\pi$ approximation $90\%$ times

In [ ]:
plot(1:k, reduce(hcat, statistics)',
    label =["Average" "Below 5%" "Top 5%"],
    xlabel = "k (10ᵏ number of simulations)",
    ylabel = "Samping disribution statistics of approx. π")
hline!([1, k],[pi, pi], label="True π")

The generated above graph shows how the accuracy of $\pi$ approximation increases with the number of simulations measure by $5\%-$ and $95\%-$ percentiles. Note that magrinal deacrease in the width of $90\%$ approximation confidence interval is getting smaller, even though the marginal increase in number of simulations is getter larger.

### Summary

Leveraging Monte Carlo Simulations and above `Julia` code we demonstrated how one can easily approximate $\pi$ number and how the number of simulations impact the accuracy of this approximation.

## B. Demonstrating the curse of dimentionality in Machine Learning

Many Machine Learning algorithms deal with highly dimentional datasets composed of dozens, thousands or millions of variables. In such a highly dimensional spaces there's a high sparsity of data points, meaning that the local neighboard of a signle data point is sparse of other data points. The higher the dimensionality, the lower number of data points in a local neighbourhood. This phoenomana is called **the curse of dimensionality** and was coined for the very first time by **Richard Bellman** in context of dynamic programming, but now it's also fully relevent in the context of Machine Learning. The problem created by the curse of dimensionality consists in the fact that having no or sparse local neighbourhood makes it less efficient to infere especially using non-parametrics, e.g. kernel, estimation techniques. Wollowing code snippets will present , you see how to:
1. Defining a function `hypersphere(n, dim)` which uses `dim` simulations to approximate the volume of `n`-dimensional hypersphere witten in `n`-dimensional hypercube, e.g. 2-dimensional hypersphere is a circle written in a square, 3-dimensional hypersphere is a sphere written in a cube,
2. Running a function `hypersphere(n, dim)` for `n = 1000000` and `dim=1`, `2`, `3` or `4` and interpret the results,
3. Drawing plots of results of `hypersphere(n, dim)` with `n = 1000000` against `dim` argument from `1` to `14` with: (1) linear and (2) exponential Y axis scale,
4. Calculating errors of simulated fractions again a true fractions for `dim = 1`, `2` or `3`,
5. Simulating sampling distribution of Mean Square Error for `dim = 2` and `3`. Checking if, Mean Square Error difference between `dim = 2` and `dim = 3` is statistically significant.

Let's start with loading necessary packages:

In [ ]:
using HypothesisTests

### B.1. Defining a function `hypersphere(n, dim)`

In [ ]:
hypersphere(n, dim)=mean(sum((2*rand(n,dim) .-1).^2,dims=2).<1)

`hypersphere(n, dim)` function is the extension of $pi$ function with additional argument being `dim` which was equal to $2$ for $pi$ function and here can be controled by `dim` input argument. The remaining logic is exactly the same as in $pi()$ function above.

### B.2. Runing a function `hypersphere(n, dim)`

In [ ]:
simulated_fractions = hypersphere.(10^6, 1:4)

We see that the portion of volume taken by a hypersphere within a hypercube deacreases significantly from $100\%$ for $1$-dimension, i.e. an interval written in the same interval, through $\approx79\%$ for a $2$-dimensions, i.e. a circle written in a square, and $52\%$ for a sphere written in a cube for $3$ dimesions, to $31\%$ for $4$-dimensional hypersphere written in a corresponding hypercube. From Machine Learning perspective, it means that building non-parametric predictive model, e.g. k-NN, will require ca. 3 more data, if 4 predictors will be employed instead of 1 predictor.

### B.3. Drawing plots of results of `hypersphere(n, dim)`

In [ ]:
dims = 14
plot(1:dims, hypersphere.(100000, 1:dims), xlim = (1,dims), layout = (2,1), legend=false,
    xlabel = "Number of dimensions", ylabel = "Fraction of Volume", title = "Linear scale")
plot!(1:dims, hypersphere.(100000, 1:dims),  xlim = (1,dims), subplot=2, 
    yaxis= :log10, title = "Exponential scale", legend = false)

Based on above plots, we see a drastic decrease of volume fraction of a hypersphere written a hypercube. As we can see on the 2nd plot with an exponential scale, this decrease is effectivaly exponential.

### B.4. Calculating errors of simulated fractions

For $1$ dimension we have an $1$-long interval written in a $1$-long interval, so a true fraction is $100\%$. Simulated fractions are also exacty to the point with $0$ error:

In [ ]:
simulated_fractions[1] - 1/1

For $2$ dimensions there's a quarter of circle with $r=1$ written in a $1 \times 1$ square. The field of such a quarter of circle is equal to $\frac{\pi}{4}$ and our simmulated fraction is quite close to this true value: 

In [ ]:
simulated_fractions[2] - pi / 4

For $3$ dimensions there's a one eighth of sphere with $r=1$ written in a $1 \times 1 \times 1 $ cube. The volume of such a one eighth sphere is equal to $\frac{4}{3} \pi / 8$ and our simmulated fraction is quite close to this true value: 

In [ ]:
simulated_fractions[3] - 4/3 * pi / 8

Moreover, you can observe that the absolute error increases with a dimension increase. You can run a few independent simulations to check it holds true most of the times or even run significant ($>1000$) number of simulations to confirm it, which will be done in the next part.

### B.5. Simulating sampling distribution of Squared Errors


To get the sampling distribution of Squared Error (SE) for `dim = 2` and `dim = 3`, we simulate $1000$ samples for each of two dimensions using `hypersphere(10 ^ 5, dim)` function. Then the results of this function is subtracted from a true fraction and taken to the square and store in `SEs` matrix: 

In [ ]:
simulations = [hypersphere(10 ^ 5, dim) for i in 1:1000, dim in 2:3]
SEs = (simulations .- [pi / 4, 4/3 * pi / 8]') .^2

Below we histogram both sampling distributions of squared errors. Even though both histogram overlaps significantly, it's still noticable that the probability mass is more to the right for higher dimension, meaning that squared errors are on average higher for a higher dimension.

In [ ]:
histogram(SEs, fillalpha=0.5, normalized=true, label=["dim = 2" "dim = 3"])

This observation is confirmed, when we calculate the Mean Squared Error for both dimensions.

In [ ]:
MSE = mean(SEs, dims=1)

We see MSE for higher dimension is higher. The question is if the difference is statistiacally significant. Therefore, we run a t-test to check it.

In [ ]:
OneSampleTTest(SEs[:,1], SEs[:,2])

Our tets null hypothesis states that Mean Square Error are equal between both samples, i.e. $H_0: MSE_{dim=2}=MSE_{dim=3}$, while the alternative hypothesis states that MSE is higher for higher dimension , i.e.  $H_1: MSE_{dim=2} < MSE_{dim=3}$. Given our `two-sided p-value` is lower than $5\%$, we recejct the null hypothesis and state that MSE for higher dimension is higher. This result is somehow to be expected exactly due to the curse of dimensionality.

## C. Constructing a statistical test indicating a trully superior player of table football match

Imagine, you're playing a table football match with your friend. Typically, the game ends when the first player scores 10 goals and become a winner. Let's assume you won 10 to 5. Try to answer following questions using simulations and statistical inference in `Julia`:
- Based on that result of winning 10 to 5, can you claim you're a superior player to your oponent or maybe you had just a luck and such a result could easily occur even if both of you play on the same level? 
- What's the minimum difference of goals to allow you claim you're a superior player?
- How this minimum goals' difference will change if we allow to play not up to 10 goals but up to 20, 50 or even 100 goals?

In this section you'll learn how to:
1. Simulating a goal difference for a single match assuming both players of the same skills' level.
2. Construct a statistical test for winner superiority.
3. Calculate a critical test value for matches played up to 100 goals.


We start with loading necessary packages for this section:

In [ ]:
using Random

### C.1. Simulating a goal difference for a single match assuming both players of the same skills' level 

We start from the assumption that both players have the same level of skills. Given that the sequence of goals can be modelled as the random sequence of bits with 50% change for each bit. The sequence is trimmed/ended when the sum of either of bits reaches 10 goals. Then we calculate the difference  between 10 goals reached by a winner and number of goals scored by an oponent. Such random difference is simulated by following `goals_difference(goals_to_win)` function, where `goals_to_win` defines how many goals are required to win a match and in a default setting we assume `goals_to_win=10`:

In [ ]:
function goals_difference(goals_to_win)
    match=bitrand(2 * goals_to_win - 1)
    (2 * goals_to_win .- filter(!isnothing, [findfirst(cumsum(.!match) .== goals_to_win), findfirst(cumsum(match) .== goals_to_win)]))[1]
end 

There are two lines of code in the body of `goals_difference(goals_to_win)` function:
1. Funciton `bitrand` simulates the random sequnce of `0` and `1` with each having 50% change of occuring. The lenght of the sequence is `2 * goals_to_win - 1` as this is the maximum lenght of the match in case in which the winner wins by the margin of just 1 goal.
2. Next, we want to find out how many goals were scored by a lost party to calculate the difference between him and the winner's number of goals. Therefore, we use `cumsum` calculating cummulative sum. THen we use `findfirst()` function to find first occurance of cummulative sum eqal to `gaols_to_win`, as this information allows us to infere about goals scored by an oponnent. For instance, if you won 10 to 5, then the first position reachint 10 gaols will be 15. Based on that, we can substract this value from 20 (`2 * goals_to_win`) to infere that 5 goals were scored by an opponent, which is returned by `goals_difference()` function.
Now, we can simmulate $10000$ goals' differences using `goals_difference()` function and plot a histogram of it. We see that for both players of the same skills, the distribution of goals difference will be cumulated around small number of goal difference equal to from 1 to 4, but it's not so rare to observe 5 or more difference of goals, even though players are equally good. 

In [ ]:
sample = [goals_difference(10) for _ in 1:10 ^ 5]
histogram(sample, legend = false, title = "Distribution of goals' difference",
            xlabel="Goals' difference", ylabel="Frequency")

### C.2. Constructing a statistical test for winner superiority
In our statistical test for winner superiority we have two hypothesis:

$H_0:$ both players are equally good.

$H_1:$ The winner is a better player.

Test statistic is a goal difference, which distribution under the true null hypothesis was simulated above in the histogram. To calculate a critical value of our test, we need to calculate 95th percentile of simulated distribution, which is equal to: 

In [ ]:
percentile(sample, 95)

The critical value of 7 means that the winner needs to win by more than 7 goals, i.e. 8, 9 or 10, to be allowed to claim her superiority over an oponent. It means also that winning 10 to 7 or less is not enough to claim it. It proves that match played up to 10 goals creates very small sample, which doesn't provide a strong signal about somebody's superior skills. Let's see how long should the game take time to have more predictive power. 

### C.3 Calculating a critical test value for matches played up to 100 goals
Since the critical test value is a percentile of simulated ditribution of goals' difference for a game played up to a specific number of goals, let's calculate it for other than 10 goals needed to win a match. We'll achive it through defining `min_signific_goal_diff()`:

In [ ]:
function min_signific_goal_diff(goals_to_win;  significance_level = 0.05, n_sim = 10000)
    percentile([goals_difference(goals_to_win) for _ in 1:n_sim], 100 * (1-significance_level))
end


Let's run this function for: 5, 10 and 20 goals required to win a match and see what critical values are:

In [ ]:
min_signific_goal_diff.([5 10 20])

1. Critial value for a match played up to 5 goals is 5 goals. It means that it's not possible to reject a null hypothesis with a significance level up to 5%. In other words, match up to 5 goals brings to little information to be able infere somebody's superiority in skills.
2. Critial value for a match played up to 10 goals is 7 goals and it confirms what we have already calculated and interpreted above. We run this to check if our calculations are correct and aligned.
3. Critial value for a match played up to 20 goals is 11 goals, which means it's enought to win by margin of 12 or more goals to claim a true superiority over an oponent.

We see that that critial value decrease relativaly to the number goals required to win a match. For 10 goals it was 7 goals (70%) and for 20 goals it was 11 goals (55%). Let's see more profoundly how critical value changes with the number of goals required to win a match, by drawing this plot:

In [ ]:
max_goals_to_win = 100
plot(1:max_goals_to_win, min_signific_goal_diff.(1:max_goals_to_win; significance_level=0.05),
    xlabel = "Goals required to win", ylabel="Critical test value", label = "Critial value",
    legend=:bottomright)
Plots.abline!(1, 0, line=:dash, label = "45-degrees line")

The plot illustrates how the critical test value changes with the number of goals required to win. We see that this blue line is concave, meaning that from each additional goal required to win the lesser proportion is required to be scored by a winner to prove her superiority in statistically significant way. Moreover, you might see (depending on a simulation seed) that blue line might decrease at some points. This is due to the sample randomness and estimation error of critial value resulted from the fact that test static is simulated on a limited sample. To improve the estimation the sample number should be increased.  

*Preparation of this workshop has been supported by the Polish National Agency for Academic Exchange under the Strategic Partnerships programme, grant number BPI/PST/2021/1/00069/U/00001.*

![SGH & NAWA](../logo.png)